# Rsymphony

Rsymphony 是一个开源的用C实现的混合整数线性规划(mixed-integer linear programs, MILPs)解决器。

## Install

In [2]:
install.packages("Rsymphony")

ERROR: Error in contrib.url(repos, "source"): trying to use CRAN without setting a mirror


## Usage

```
Rsymphony_solve_LP(obj, mat, dir, rhs, bounds = NULL, 
                   types = NULL, max = FALSE, verbosity = -2, 
                   time_limit = -1, node_limit = -1, gap_limit = -1, 
                   first_feasible = FALSE,
                   write_lp = FALSE, write_mps = FALSE)
```

### Arguments

arg         | desc
------------|--------------------------------------------------------------------------------
obj         | a vector with the objective coefficients
mat         | a vector or a matrix of the constraint coefficients
dir         | a character vector with the directions of the constraints. Each element must be one of "<", "<=", ">", ">=", "==" or "!=".
rhs         | the right hand side of the constraints
bounds      | NULL (default) or a list with elements `upper` and `lower` containing the indices and corresponding bounds of the objective variables. The default for each variable is a bound between 0 and Inf.
types       | a character vector giving the types of the objective variables, with "C", "I", and "B" corresponding to continuous, integer, and binary, respectively, or NULL (default), taken as all-continuous. Recycled as needed.
max         | a logical giving the direction of the optimization. TRUE means that the objective is to maximize the objective function, FALSE (default) means to minimize it.
verbosity   | an integer defining the level of verbosity, -2 (default) means no output.
time_limit  | an integer defining the time limit in seconds, -1 (default) means no time limit
node_limit  | an integer defining the limit in number of iterations, -1 (default) means no node limit
gap_limit   | when the gap between the lower and the upper bound reaches this point, the solution process will stop and the best solution found to that point will be returned, -1 (default) means no gap limit.
first_feasible | a logical defining if the solution process should stop after the first feasible solution has been found, FALSE (default) means that the solution process does not stop after the first feasible solution has been found.
write_lp    | a logical value indicating if an LP representation of the problem should be written for debugging purposes, FALSE (default) means no LP file is written.
write_mps   | a logical value indicating if an MPS representation of the problem should be written for debugging purposes, FALSE (default) means no MPS file is written.

### Value

value     | desc
----------|------------------------------------------------------------------------------
solution  | the vector of optimal coefficients
objval    | the value of the objective function at the optimum
status    |an integer with status information about the solution returned: 0 if the optimal solution was found, a non-zero value otherwise.

## Example

### example 1

$$
max( z=3 x_1 + 1 x_2 + 3 x_3)\\
\left\{ 
\begin{aligned}
  3 x_1 + 4 x_2 + 2 x_3 &<= 60\\
  2 x_1 +   x_2 +   x_3 &<= 40\\
  x_1 + 3 x_2 + 2 x_3 &<= 80\\
  x_1, x_2, x_3 为非负实数
\end{aligned}
\right.
$$

In [3]:
library(Rsymphony)

obj <- c(2, 4, 3)
mat <- matrix(c(3, 2, 1, 4, 1, 3, 2, 1, 2), nrow = 3)
mat

dir <- c("<=", "<=", "<=")
rhs <- c(60, 40, 80)
max <- T

Rsymphony_solve_LP(obj, mat, dir, rhs, max = max)

3,4,2
2,1,1
1,3,2


$solution
[1]  0  0 30

$objval
[1] 90

$status
TM_OPTIMAL_SOLUTION_FOUND 
                        0

### example 2

$$
max(3 x_1 + 1 x_2 + 3 x_3)\\
\left\{ 
\begin{aligned}
-1 x_1 + 2 x_2 +   x_3 &<= 4\\
      4 x_2 - 3 x_3 &<= 2\\
   x_1 - 3 x_2 + 2 x_3 &<= 3\\
x_1, x_3 为非负整数\\
x_2 为非负实数
\end{aligned}
\right.
$$

In [5]:
obj <- c(3, 1, 3)
mat <- matrix(c(-1, 0, 1, 2, 4, -3, 1, -3, 2), nrow = 3)
mat

dir <- c("<=", "<=", "<=")
rhs <- c(4, 2, 3)
max <- TRUE
types <- c("I", "C", "I")

Rsymphony_solve_LP(obj, mat, dir, rhs, types = types, max = max)

-1,2,1
0,4,-3
1,-3,2


$solution
[1] 5.00 2.75 3.00

$objval
[1] 26.75

$status
TM_OPTIMAL_SOLUTION_FOUND 
                        0

### example 3

$$
\left\{ 
\begin{aligned}
  -Inf < &x_1 <= 4   \\
   0 <= &x_2 <= 100 \\
   2 <= &x_3 < Inf  \\
\end{aligned}
\right.
$$

In [6]:
bounds <- list(lower = list(ind = c(1L, 3L), val = c(-Inf, 2)), 
               upper = list(ind = c(1L, 2L), val = c(4, 100)))
Rsymphony_solve_LP(obj, mat, dir, rhs, types = types, max = max, bounds = bounds)

$solution
[1] 4.0 2.5 3.0

$objval
[1] 23.5

$status
TM_OPTIMAL_SOLUTION_FOUND 
                        0